In [6]:
import tensorflow as tf
import numpy as np

def generate_CAM(model, img_tensor, class_index):
    # Get the output tensor of the last convolutional layer
    last_conv_layer = model.get_layer('last_conv_layer')
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

    # Get the gradients of the predicted class with regard to the output feature map of the last conv layer
    with tf.GradientTape() as tape:
        preds = model(img_tensor)
        class_activation = preds[:, class_index]

    grads = tape.gradient(class_activation, last_conv_layer_model.output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Compute the CAM
    cam = tf.reduce_sum(tf.multiply(pooled_grads, last_conv_layer_model.output), axis=-1)
    cam = tf.nn.relu(cam)

    # Normalize the CAM
    cam = (cam - tf.reduce_min(cam)) / (tf.reduce_max(cam) - tf.reduce_min(cam))
    return cam.numpy()


In [7]:
import cv2

def overlay_CAM_on_image(spectrogram_img, cam, threshold=0.5):
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap[np.where(cam <= threshold)] = 0

    # Resize heatmap to match the original spectrogram's dimensions
    heatmap = cv2.resize(heatmap, (spectrogram_img.shape[1], spectrogram_img.shape[0]))

    # Overlay the heatmap on the original spectrogram
    overlaid_img = cv2.addWeighted(spectrogram_img, 0.7, heatmap, 0.3, 0)

    return overlaid_img


In [8]:
import matplotlib.pyplot as plt

def visualize_CAM_on_spectrogram(model, spectrogram_img, class_index):
    # Preprocess the input spectrogram image (e.g., normalize it to [0, 1])
    img_tensor = preprocess_spectrogram_image(spectrogram_img)

    # Generate the CAM for the specified class
    cam = generate_CAM(model, img_tensor, class_index)

    # Overlay CAM on the spectrogram image
    overlaid_img = overlay_CAM_on_image(spectrogram_img, cam)

    # Display the original spectrogram and the CAM overlay
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(spectrogram_img, cmap='viridis')
    plt.title('Mel Spectrogram')
    plt.subplot(1, 2, 2)
    plt.imshow(overlaid_img)
    plt.title('CAM Overlay')
    plt.show()


In [1]:
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import matplotlib.pyplot as plt

# Load the trained model
model = load_model('Model1')  # Replace with the path to your trained model

# Define a function to extract Mel spectrogram features from audio files
def extract_mel_spectrogram(file_path, n_mels=128):
    audio, sr = librosa.load(file_path, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
    return mel_spec

# Load an example audio file and extract its Mel spectrogram
audio_file_path = 'cat_3.wav'  # Replace with the path to your audio file
mel_spectrogram = extract_mel_spectrogram(audio_file_path)

# Resize the Mel spectrogram to match the model's input size
model_input_shape = model.layers[0].input_shape[0][1:]  # Get the expected input shape
mel_spectrogram = cv2.resize(mel_spectrogram, model_input_shape)

# Normalize the Mel spectrogram
mel_spectrogram = (mel_spectrogram - np.min(mel_spectrogram)) / (np.max(mel_spectrogram) - np.min(mel_spectrogram))

# Expand dimensions to match the model's input shape
mel_spectrogram = np.expand_dims(mel_spectrogram, axis=-1)
mel_spectrogram = np.expand_dims(mel_spectrogram, axis=0)

# Get model predictions
predictions = model.predict(mel_spectrogram)
predicted_class = np.argmax(predictions)

# Generate the Class Activation Map (CAM)
last_conv_layer = model.get_layer('conv2d')  # Replace with the name of the last convolutional layer
grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(mel_spectrogram)
    loss = predictions[:, predicted_class]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = gate_f * gate_r * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))
cam = np.dot(output, weights)

# Normalize the CAM
cam = np.maximum(cam, 0)
cam = cam / cam.max()

# Resize the CAM to match the Mel spectrogram dimensions
cam = cv2.resize(cam, (mel_spectrogram.shape[2], mel_spectrogram.shape[1]))

# Create a heatmap
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

# Overlay the heatmap on the original Mel spectrogram
heatmap = cv2.resize(heatmap, (mel_spectrogram.shape[2], mel_spectrogram.shape[1]))
output_image = cv2.addWeighted(cv2.cvtColor(mel_spectrogram[0], cv2.COLOR_GRAY2BGR), 0.5, heatmap, 0.5, 0)

# Find contours in the CAM
contours, _ = cv2.findContours(np.uint8(255 * cam), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around vocalization regions
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the original Mel spectrogram with CAM and bounding boxes
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(mel_spectrogram[0, :, :, 0], cmap='viridis')
plt.title('Mel Spectrogram')
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title('CAM Overlay with Bounding Boxes')
plt.show()


2023-09-21 11:32:13.640663: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-09-21 11:32:13.640681: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-21 11:32:13.640685: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-21 11:32:13.640847: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-21 11:32:13.640863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TypeError: 'NoneType' object is not subscriptable